In [1]:
import numpy as np
import matplotlib.pyplot as plt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import pandas as pd
import os
import math
import subprocess

## Test functions

In [2]:
def hump_nf(*args):
    x = args[0][0]
    y = args[0][1]
    result = (4 - 2.1*x**2 + x**4/3)*x**2 + x*y + (-4 + 4*y**2)*y**2
    result = float(result)
    now = np.array([x, y], dtype=np.float)
    point1 = np.array([0.0898, -0.7126], dtype=np.float)
    point2 = np.array([-0.0898, 0.7126], dtype=np.float)
    min_dist = min(((point1 - now)**2).sum()**.5, ((point2 - now)**2).sum()**.5)
    return {'loss': result, 'status': STATUS_OK, 'dist': min_dist, 'pure_loss': result}

In [3]:
def hump_noisy(*args):
    x = args[0][0]
    y = args[0][1]
    result = (4 - 2.1*x**2 + x**4/3)*x**2 + x*y + (-4 + 4*y**2)*y**2
    result = float(result)
    now = np.array([x, y], dtype=np.float)
    point1 = np.array([0.0898, -0.7126], dtype=np.float)
    point2 = np.array([-0.0898, 0.7126], dtype=np.float)
    min_dist = min(((point1 - now)**2).sum()**.5, ((point2 - now)**2).sum()**.5)
    noise = np.random.normal() * 0.1

    return {'loss': result + noise, 'status': STATUS_OK, 'dist': min_dist, 'pure_loss': result, 'loss_variance':0.1}

In [4]:
def branin_nf(*args):
    x = args[0][0]
    y = args[0][1]
    result = np.square(y - (5.1/(4*np.square(math.pi)))*np.square(x) + 
         (5/math.pi)*x - 6) + 10*(1-(1./(8*math.pi)))*np.cos(x) + 10
    result = float(result)
    now = np.array([x, y], dtype=np.float)
    point1 = np.array([-np.pi, 12.275], dtype=np.float)
    point2 = np.array([np.pi, 2.275], dtype=np.float)
    point3 = np.array([9.42478, 2.475], dtype=np.float)
    min_dist = min(((point1 - now)**2).sum()**.5, ((point2 - now)**2).sum()**.5)
    min_dist = min(min_dist, ((point3 - now)**2).sum()**.5)
    return {'loss': result, 'status': STATUS_OK, 'dist': min_dist, 'pure_loss': result}

In [5]:
def branin_noisy(*args):#
    x = args[0][0]
    y = args[0][1]
    result = np.square(y - (5.1/(4*np.square(math.pi)))*np.square(x) + 
         (5/math.pi)*x - 6) + 10*(1-(1./(8*math.pi)))*np.cos(x) + 10
    
    result = float(result)
    now = np.array([x, y], dtype=np.float)
    point1 = np.array([-np.pi, 12.275], dtype=np.float)
    point2 = np.array([np.pi, 2.275], dtype=np.float)
    point3 = np.array([9.42478, 2.475], dtype=np.float)
    min_dist = min(((point1 - now)**2).sum()**.5, ((point2 - now)**2).sum()**.5)
    min_dist = min(min_dist, ((point3 - now)**2).sum()**.5)
    noise = np.random.normal() * 0.1
    return {'loss': result + noise, 'status': STATUS_OK, 'dist': min_dist, 'pure_loss': result, 'loss_variance':0.1}

In [6]:
def griewank_nf(*args):
    x = np.array(args[0])
    result = 1 + 1.0/4000 * (x**2).sum() - np.prod(np.cos(x/np.arange(1, x.size + 1)**0.5))
    result = float(result)
    min_dist = (x ** 2).sum()**.5
    return {'loss': result, 'status': STATUS_OK, 'dist': min_dist, 'pure_loss': result}

In [7]:
def griewank_noisy(*args):
    x = np.array(args[0])
    result = 1 + 1.0/4000 * (x**2).sum() - np.prod(np.cos(x/np.arange(1, x.size + 1)**.5))
    result = float(result)
    min_dist = (x**2).sum()**.5
    noise = np.random.normal() * 0.1
    return {'loss': result + noise, 'status': STATUS_OK, 'dist': min_dist, 'pure_loss': result, 'loss_variance':0.1}

In [8]:
def rosenbrock_nf(*args):
    x = args[0][0]
    y = args[0][1]
    result = 100*(y - x**2)**2 + (x - 1)**2
    result = float(result)
    now = np.array([x, y], dtype=np.float)
    point1 = np.array([1, 1], dtype=np.float)
    min_dist = ((point1 - now)**2).sum()**.5
    return {'loss': result, 'status': STATUS_OK, 'dist': min_dist, 'pure_loss': result,}

In [9]:
def rosenbrock_noisy(*args):#
    x = args[0][0]
    y = args[0][1]
    result = 100*(y - x**2)**2 + (x - 1)**2
    result = float(result)
    now = np.array([x, y], dtype=np.float)
    point1 = np.array([1, 1], dtype=np.float)
    min_dist = ((point1 - now)**2).sum()**.5
    noise = np.random.normal() * 0.1
    return {'loss': result + noise, 'status': STATUS_OK, 'dist': min_dist, 'pure_loss': result, 'loss_variance':0.1}

In [10]:
def learn_svm(*args):
    C, gamma = args[0]
    command = ['../experiments_results/SVC_MNIST/learn.sh',
               '--gamma', str(float(gamma)), '--C', str(float(C))]
    p = subprocess.Popen(command, stdout=subprocess.PIPE)
    result, _ = p.communicate()
    result = float(result)
    print('Result = %f' % result)
    return {'loss': -result, 'status': STATUS_OK, 'pure_loss': -result}

## Experiments: acquisition function EI

In [11]:
# function, space to optimize
params = [(hump_nf,           [hp.uniform('x', -3, 3), hp.uniform('y', -2, 2)]),
          (hump_noisy,        [hp.uniform('x', -3, 3), hp.uniform('y', -2, 2)]),
          (branin_nf,         [hp.uniform('x', -5, 10), hp.uniform('y', 0, 15)]),
          (branin_noisy,      [hp.uniform('x', -5, 10), hp.uniform('y', 0, 15)]),
          (rosenbrock_nf,     [hp.uniform('x', -5, 10), hp.uniform('y', -5, 10)]),
          (rosenbrock_noisy,  [hp.uniform('x', -5, 10), hp.uniform('y', -5, 10)]),
          (griewank_nf,       [hp.uniform('x1', -2, 2), hp.uniform('x2', -2, 2),
                               hp.uniform('x3', -2, 2), hp.uniform('x4', -2, 2),
                               hp.uniform('x5', -2, 2), hp.uniform('x6', -2, 2)]),
          (griewank_noisy,    [hp.uniform('x1', -2, 2), hp.uniform('x2', -2, 2), \
                               hp.uniform('x3', -2, 2), hp.uniform('x4', -2, 2), \
                               hp.uniform('x5', -2, 2), hp.uniform('x6', -2, 2)]),
          (learn_svm,         [hp.uniform('C', 1, 10000), hp.uniform('gamma', 1e-6, 1e-1)])
         ]

In [12]:
out_dir = os.path.join('results')
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

# with gaussian noise, sigma = 0.01
for func, space in params:
    trials = Trials()
    best = fmin(func, space=space, algo=tpe.suggest, max_evals=40,
                rstate=np.random.RandomState(1), trials=trials)
    key = list(trials.idxs_vals[0].keys())[0]
    tpe_results = pd.DataFrame({'loss': [x['pure_loss'] for x in trials.results], 
                                'iteration': trials.idxs_vals[0][key],
                                **trials.idxs_vals[1]})
    tpe_results = tpe_results.set_index('iteration')
    tpe_results.to_csv(os.path.join(out_dir, func.__name__)+'.csv')

100%|██████████| 40/40 [00:00<00:00, 186.40trial/s, best loss: 0.34227903451537073]
Result = 0.955800                                     
Result = 0.926100                                                       
Result = 0.935100                                                       
Result = 0.920100                                                       
Result = 0.933800                                                       
Result = 0.892900                                                       
Result = 0.857300                                                       
Result = 0.937900                                                       
Result = 0.935200                                                       
Result = 0.944000                                                       
Result = 0.952400                                                        
Result = 0.925100                                                        
Result = 0.913600                                       

In [24]:
out_dir = os.path.join('results')
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

# with gaussian noise, sigma = 0.1
for func, space in params[:-1]:
    trials = Trials()
    best = fmin(func, space=space, algo=tpe.suggest, max_evals=40,
                rstate=np.random.RandomState(1), trials=trials)
    key = list(trials.idxs_vals[0].keys())[0]
    tpe_results = pd.DataFrame({'loss': [x['pure_loss'] for x in trials.results], 
                                'iteration': trials.idxs_vals[0][key],
                                **trials.idxs_vals[1]})
    tpe_results = tpe_results.set_index('iteration')
    tpe_results.to_csv(os.path.join(out_dir, func.__name__)+'.csv')

100%|██████████| 40/40 [00:00<00:00, 186.35trial/s, best loss: 0.3391296940054707]


In [24]:
local_minima = [
          (hump_nf,           (np.array([0.0898, -0.7126], dtype=np.float),
                               np.array([-0.0898, 0.7126], dtype=np.float)), ('x', 'y')),
          (hump_noisy,        (np.array([0.0898, -0.7126], dtype=np.float),
                               np.array([-0.0898, 0.7126], dtype=np.float)), ('x', 'y')),
          (branin_nf,         (np.array([-np.pi, 12.275], dtype=np.float),
                               np.array([np.pi, 2.275], dtype=np.float),
                               np.array([9.42478, 2.475], dtype=np.float)), ('x', 'y')),
          (branin_noisy,      (np.array([-np.pi, 12.275], dtype=np.float),
                               np.array([np.pi, 2.275], dtype=np.float),
                               np.array([9.42478, 2.475], dtype=np.float)), ('x', 'y')),
          (rosenbrock_nf,     (np.array([1, 1], dtype=np.float),), ('x', 'y')),
          (rosenbrock_noisy,  (np.array([1, 1], dtype=np.float),), ('x', 'y')),
          (griewank_nf,       (np.zeros(6, dtype=np.float,)), ('x1', 'x2', 'x3', 'x4', 'x5', 'x6')),
          (griewank_noisy,    (np.zeros(6, dtype=np.float,)), ('x1', 'x2', 'x3', 'x4', 'x5', 'x6')),
          (learn_svm,         (None, ), ('C', 'gamma'))
         ]

In [35]:
out_dir = os.path.join('results')
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

n_calls = 40
for func, pts, names in local_minima:
    tpe_results = pd.read_csv(os.path.join(out_dir, func.__name__)+'.csv')
    if func != learn_svm:
        dist = []
        for i in range(n_calls):
            x = []
            for name in names:
                x.append(tpe_results.loc[i, name])
            x = np.array(x, dtype=np.float)
            min_dst = np.linalg.norm(x - pts[0])
            for k in range(1, len(pts)):
                min_dst = min(min_dst, np.linalg.norm(x - pts[k]))
            dist.append(min_dst)
    else:
        dist = np.zeros_like(n_calls)
    tpe_results['dist'] = dist
    tpe_results[['iteration', 'loss', 'dist', *names]].set_index('iteration').to_csv(os.path.join(out_dir, func.__name__)+'.csv')
    break

In [34]:
tpe_results[['iteration', 'loss', 'dist', *names]]

,iteration,loss,dist,x,y
0,0,1.582502,1.027073,1.062687,-1.041797
1,1,34.483312,2.542178,-2.630090,0.810580
2,2,0.498394,0.902820,-0.992361,0.734230
3,3,89.250717,2.894301,-2.919306,1.321596
4,4,-0.359492,0.452222,-0.530562,0.813760
5,5,12.334951,1.097374,-0.842928,1.510741
6,6,34.798146,1.201463,-0.411247,1.870263
7,7,15.964168,2.332660,-2.419686,0.598872
8,8,0.197327,1.343670,1.433276,-0.735418
9,9,-0.143258,0.502756,0.111311,-0.210304
